In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from collections import defaultdict

In [2]:
# Load the CSV file (whitespace-delimited)
csv_path = "C:\\Users\\aditi\\Downloads\\dbmopp_dataset_perf.csv"
df = pd.read_csv(csv_path, sep='\s+')

In [3]:
# Set budgets and bootstrap parameters
budgets = [5000, 10000, 30000, 50000]
n_iterations = 1000
noise_std = 1e-4  # Standard deviation for Gaussian noise

In [4]:
# Store results for all budgets
all_results = []

In [5]:

# Loop over each budget
for budget_level in budgets:
    df_budget = df[df['budget'] == budget_level]
    if df_budget.empty:
        print(f"No data for budget {budget_level}, skipping...")
        continue

    algo_rank_counts = defaultdict(lambda: [0] * 4)  # 4 possible ranks

    for _ in range(n_iterations):
        # Bootstrap resample
        sample = resample(df_budget, replace=True, n_samples=len(df_budget))

        # Mean hypervolume per algorithm
        mean_scores = sample.groupby('algo')['hypervolume'].mean().reset_index()

        # Rank algorithms by descending hypervolume
        ranked = mean_scores.sort_values(by='hypervolume', ascending=False).reset_index(drop=True)

        for i, row in ranked.iterrows():
            algo_rank_counts[row['algo']][i] += 1

    # Build results for this budget
    algorithms = sorted(algo_rank_counts.keys())
    result = {
        'Budget': [budget_level] * len(algorithms),
        'Algorithm': algorithms
    }

    for rank in range(4):
        result[f'Rank {rank+1} Frequency'] = [
            algo_rank_counts[algo][rank] / n_iterations for algo in algorithms
        ]

    all_results.append(pd.DataFrame(result))

# Combine and reset index
final_df = pd.concat(all_results, ignore_index=True)
final_df.drop_duplicates(inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Group by budget and show clean summary
for budget in sorted(final_df['Budget'].unique()):
    print(f"\n Results for Budget = {budget}")
    display(final_df[final_df['Budget'] == budget])




 Results for Budget = 5000


,Budget,Algorithm,Rank 1 Frequency,Rank 2 Frequency,Rank 3 Frequency,Rank 4 Frequency
0,5000,IBEA,1.0,0.0,0.0,0.0
1,5000,MOEAD,0.0,1.0,0.0,0.0
2,5000,NSGAII,0.0,0.0,1.0,0.0
3,5000,Random,0.0,0.0,0.0,1.0



 Results for Budget = 10000


,Budget,Algorithm,Rank 1 Frequency,Rank 2 Frequency,Rank 3 Frequency,Rank 4 Frequency
4,10000,IBEA,1.0,0.0,0.0,0.0
5,10000,MOEAD,0.0,0.0,1.0,0.0
6,10000,NSGAII,0.0,1.0,0.0,0.0
7,10000,Random,0.0,0.0,0.0,1.0



 Results for Budget = 30000


,Budget,Algorithm,Rank 1 Frequency,Rank 2 Frequency,Rank 3 Frequency,Rank 4 Frequency
8,30000,IBEA,1.0,0.0,0.0,0.0
9,30000,MOEAD,0.0,0.0,0.0,1.0
10,30000,NSGAII,0.0,1.0,0.0,0.0
11,30000,Random,0.0,0.0,1.0,0.0



 Results for Budget = 50000


,Budget,Algorithm,Rank 1 Frequency,Rank 2 Frequency,Rank 3 Frequency,Rank 4 Frequency
12,50000,IBEA,1.0,0.0,0.0,0.0
13,50000,MOEAD,0.0,0.0,0.0,1.0
14,50000,NSGAII,0.0,1.0,0.0,0.0
15,50000,Random,0.0,0.0,1.0,0.0


None
